In [22]:
"""
Author:         Shraey Bhatia
Date:           October 2016
File:           doc2vectrain.py

Gives a trained document vectors model (also known as Doc2VecModel). 
The input format are documents extrated using wiki extractor and tokenized using stanford tokenizer stored in a directory
Output would be a trained doc2vec model.
Parameters taken for main_train.py
"""

import os
import re
import sys
import gensim
import argparse
import codecs
import unicodedata
import multiprocessing
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import logging

In [23]:
#These are the arguments that are passed in main_train.py 
parser = argparse.ArgumentParser()
parser.add_argument("epochs")        # Number of training epochs
parser.add_argument("input_dir")     
parser.add_argument("output_dir")
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] epochs input_dir output_dir
ipykernel_launcher.py: error: the following arguments are required: input_dir, output_dir


SystemExit: 2

/home/andreas/bin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# Checks if the output directory specified already exists. If it does remove it.

if os.path.isdir(args.output_dir):
    del_query = "rm -r "+'outdir'
    os.system(del_query)

# Create an output directory for the model
#query = "mkdir "+args.output_dir
query = "mkdir "+'outdir'
os.system(query)

NameError: name 'args' is not defined

In [10]:
from constants import *
output = os.path.join(TMP_PATH, "doc2vecmodel.d2v")

# Doc2vec Input documents Class. It uses yield to optimize memory usage and "tag" is Document title with an undescore.
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

    def __iter__(self):

        for source in self.sources:
            with codecs.open(source, "r", "utf-8") as fin:
                for cnt,line in enumerate(fin):
                    if "<doc" in line:           # Every new document starts with this format
                        found = ""

                        m = re.search('title="(.*)">',line)    # This gives the document title of Wikipedia
                        if m:
                            found = m.group(1)
                            #found = found.lower()
                            found = unicodedata.normalize("NFKD", found) 
                            found = found.replace(" ","_") 
                            found = found.encode('utf-8')

                        else:
                            found = ""
                        values = []
                    else:
                        if "</doc" not in line:                      #</doc tells us end of document, till not reached it is same document
                            for word in line.split(" "):
                                values.append(word.strip())
                        if "</doc" in line:
                            if found!= "":
                            
                                yield LabeledSentence(words = values, tags = [found])
                                
cores = multiprocessing.cpu_count() 
print(cores)
filenames=[]

8


In [13]:
# Get all tokenised files in directory and subdirectories ( tokenised by stanford parser)
input_dir = os.path.join(DATA_BASE, 'docs_tokenised')
for path,subdirs,files in os.walk(input_dir):
    for name in files:
        temp = os.path.join(path, name)    
        filenames.append(temp) 
        
filenames

['../data/docs_tokenised/AA/wiki_07',
 '../data/docs_tokenised/AA/wiki_08',
 '../data/docs_tokenised/AA/wiki_01',
 '../data/docs_tokenised/AA/wiki_06',
 '../data/docs_tokenised/AA/wiki_03',
 '../data/docs_tokenised/AA/wiki_04',
 '../data/docs_tokenised/AA/wiki_10',
 '../data/docs_tokenised/AA/wiki_05',
 '../data/docs_tokenised/AA/wiki_00',
 '../data/docs_tokenised/AA/wiki_09',
 '../data/docs_tokenised/AA/wiki_02']

In [18]:
sentences = LabeledLineSentence(filenames)

In [17]:
for i, s in enumerate(sentences):
    s
i

/home/andreas/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


2215484

In [20]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#Doc2Vec model initialization and parameters 
model = Doc2Vec(vector_size=300, window=15, min_count=20, sample=1e-5, workers=cores, hs=0, dm=0, negative=5, dbow_words=1, dm_concat=0)
model.build_vocab(sentences)

#Model Training 
for epoch in range(int(args.epochs)):
    model.train(sentences, total_examples=2215484)
    print("Epoch completed: "+str(epoch+1))

model.save(output)

/home/andreas/bin/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-10-10 01:59:54,859 : INFO : collecting all words and their counts
/home/andreas/bin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
2018-10-10 01:59:54,860 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-10-10 01:59:57,106 : INFO : PROGRESS: at example #10000, processed 2724686 words (1213376/s), 205860 word types, 10000 tags
2018-10-10 01:59:59,519 : INFO : PROGRESS: at example #20000, processed 5663655 words (1218480/s), 344754 word types, 20000 tags
2018-10-10 02:00:01,955 : INFO : PROGRESS: at example #30000, processe

2018-10-10 02:02:40,188 : INFO : PROGRESS: at example #580000, processed 201484091 words (1141226/s), 3808673 word types, 579938 tags
2018-10-10 02:02:45,294 : INFO : PROGRESS: at example #590000, processed 207745002 words (1226486/s), 3895537 word types, 589932 tags
2018-10-10 02:02:49,946 : INFO : PROGRESS: at example #600000, processed 213638276 words (1266955/s), 3978489 word types, 599923 tags
2018-10-10 02:02:54,588 : INFO : PROGRESS: at example #610000, processed 219591473 words (1282726/s), 4058848 word types, 609921 tags
2018-10-10 02:02:59,022 : INFO : PROGRESS: at example #620000, processed 225184383 words (1261466/s), 4135294 word types, 619912 tags
2018-10-10 02:03:03,409 : INFO : PROGRESS: at example #630000, processed 230673972 words (1251505/s), 4207912 word types, 629906 tags
2018-10-10 02:03:08,188 : INFO : PROGRESS: at example #640000, processed 236841385 words (1290658/s), 4287261 word types, 639904 tags
2018-10-10 02:03:12,790 : INFO : PROGRESS: at example #650000,

2018-10-10 02:05:49,654 : INFO : PROGRESS: at example #1190000, processed 432913023 words (1217047/s), 6333309 word types, 1189619 tags
2018-10-10 02:05:52,969 : INFO : PROGRESS: at example #1200000, processed 436896865 words (1201701/s), 6371167 word types, 1199610 tags
2018-10-10 02:05:56,121 : INFO : PROGRESS: at example #1210000, processed 440626026 words (1183670/s), 6406729 word types, 1209596 tags
2018-10-10 02:05:59,320 : INFO : PROGRESS: at example #1220000, processed 444516229 words (1215964/s), 6442918 word types, 1219585 tags
2018-10-10 02:06:02,408 : INFO : PROGRESS: at example #1230000, processed 448153071 words (1178019/s), 6476143 word types, 1229579 tags
2018-10-10 02:06:05,362 : INFO : PROGRESS: at example #1240000, processed 451690387 words (1197603/s), 6509373 word types, 1239567 tags
2018-10-10 02:06:08,302 : INFO : PROGRESS: at example #1250000, processed 455303178 words (1229449/s), 6542394 word types, 1249562 tags
2018-10-10 02:06:11,720 : INFO : PROGRESS: at ex

2018-10-10 02:09:23,794 : INFO : PROGRESS: at example #1800000, processed 699548234 words (1217189/s), 8586039 word types, 1798991 tags
2018-10-10 02:09:26,458 : INFO : PROGRESS: at example #1810000, processed 702806130 words (1223420/s), 8611092 word types, 1808986 tags
2018-10-10 02:09:29,124 : INFO : PROGRESS: at example #1820000, processed 706100126 words (1235625/s), 8634997 word types, 1818981 tags
2018-10-10 02:09:31,743 : INFO : PROGRESS: at example #1830000, processed 709309306 words (1225790/s), 8658286 word types, 1828976 tags
2018-10-10 02:09:34,343 : INFO : PROGRESS: at example #1840000, processed 712454652 words (1209987/s), 8681124 word types, 1838967 tags
2018-10-10 02:09:36,881 : INFO : PROGRESS: at example #1850000, processed 715527626 words (1211165/s), 8703905 word types, 1848963 tags
2018-10-10 02:09:39,507 : INFO : PROGRESS: at example #1860000, processed 718644369 words (1187065/s), 8727316 word types, 1858960 tags
2018-10-10 02:09:42,098 : INFO : PROGRESS: at ex

KeyboardInterrupt: 

In [24]:
for epoch in range(int(args.epochs)):
    print(epoch)

NameError: name 'args' is not defined